In [ ]:
# Crash on purpose to get more ram :
#import torch
#torch.tensor([10.]*10000000000)

Let's install [PyTorch/XLA](https://github.com/pytorch/xla) which enables PyTorch on TPU. Make sure you install the nightly version, as the trainer breaks on other versions.

In [1]:
VERSION = "nightly"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4264  100  4264    0     0  60914      0 --:--:-- --:--:-- --:--:-- 60914
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
Uninstalling torch-1.5.0+cu101:
Done updating TPU runtime: <Response [200]>
  Successfully uninstalled torch-1.5.0+cu101
Uninstalling torchvision-0.6.0+cu101:
  Successfully uninstalled torchvision-0.6.0+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly-cp36-cp36m-linux_x86_64.whl...
- [1 files][106.7 MiB/106.7 MiB]                                                
Operation completed over 1 objects/106.7 MiB.                                    
Copying gs://tpu-pytorch/wheels/torch_xla-nightly-cp36-cp36m-linux_x86_64.whl...
- [1 files][122.0 MiB/122.0 MiB]                                                
Operation completed over 1 objects/122.0 MiB.         

Install transformers and the nlp package. Restart colab after this

In [2]:
!git clone https://github.com/huggingface/transformers.git
!pip install ./transformers


Cloning into 'transformers'...
remote: Enumerating objects: 31865, done.
remote: Total 31865 (delta 0), reused 0 (delta 0), pack-reused 31865
Receiving objects: 100% (31865/31865), 31.74 MiB | 35.64 MiB/s, done.
Resolving deltas: 100% (22144/22144), done.
Processing ./transformers
     |████████████████████████████████| 3.0MB 3.5MB/s 
     |████████████████████████████████| 1.1MB 40.3MB/s 
     |████████████████████████████████| 890kB 46.5MB/s 
  Created wheel for transformers: filename=transformers-2.11.0-cp36-none-any.whl size=734076 sha256=634119d824b81324fd66926845ccc0b2397b37b93f6bade9582be8fc59b9ef6b
  Stored in directory: /tmp/pip-ephem-wheel-cache-vvbvcihv/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=d388d87a387342ee9ccd829ffd19e89b0c084e3acc3646b297bc31ab829f4b60
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d

## Load and process data

In [3]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from transformers import XLMRobertaTokenizer, XLMRobertaModel
import torch.nn as nn 
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import json
import pandas as pd

import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

In [4]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

In [5]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"doanquanvietnamca","key":"2bc9ebe82282c4fe649204ee312360d1"}'}

In [6]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!pip install kaggle
!kaggle competitions download -c tweet-sentiment-extraction

  0% 0.00/41.4k [00:00<?, ?B/s]
100% 41.4k/41.4k [00:00<00:00, 39.1MB/s]
  0% 0.00/307k [00:00<?, ?B/s]
100% 307k/307k [00:00<00:00, 93.8MB/s]
  0% 0.00/1.23M [00:00<?, ?B/s]
100% 1.23M/1.23M [00:00<00:00, 83.6MB/s]


In [7]:
!unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [8]:
!mkdir data
!mv *.csv data
!mkdir models

In [9]:
import pandas as pd
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [10]:
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [11]:
train[train['text']=='']

,textID,text,selected_text,sentiment


In [12]:
train['sentiment'].value_counts()

neutral     11118
positive     8582
negative     7781
Name: sentiment, dtype: int64

In [13]:
content = []
for i in range(len(train)):
  content.append("context: %s content: %s </s>"%(train['text'][i], train['selected_text'][i]))
train['content'] = content

In [14]:
def pre(t):
  return "%s </s>"%t
train['sentiment'] = train['sentiment'].apply(pre)

In [15]:
from sklearn.model_selection import train_test_split
train = train[['content','sentiment']]
train.columns = ['text', 'target']
train, valid = train_test_split(train, test_size=0.2, random_state=42)

In [40]:
from torch.utils.data import *
class ToxicDataset(Dataset):
  def __init__(self, tokenizer, df,  max_len=512):
    self.data_column = df["text"].values
    self.class_column = df['target'].values
    self.max_len = max_len
    self.tokenizer = tokenizer
    self.inputs = []
    self.targets = []
    self._build()
  def __len__(self):
    return len(self.inputs)
  
  def __getitem__(self, index):
    source_ids = self.inputs[index]["input_ids"].squeeze()
    target_ids = self.targets[index]["input_ids"].squeeze()

    src_mask    = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
    target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

    return {"input_ids": source_ids, "attention_mask": src_mask, "target_ids": target_ids, "target_attention_mask": target_mask}
  
  def _build(self):
    for idx in range(len(self.data_column)):
      input_ =  self.data_column[idx]
      target =  self.class_column[idx]
       # tokenize inputs
      tokenized_inputs = self.tokenizer.encode_plus(
          str(input_), max_length=self.max_len, pad_to_max_length=True,truncation=True, return_tensors="pt"
      )
      tokenized_targets = self.tokenizer.encode_plus(
          str(target), max_length=2, pad_to_max_length=True, truncation=True, return_tensors="pt"
      )
      self.inputs.append(tokenized_inputs)
      self.targets.append(tokenized_targets)


In [41]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
train_dataset = ToxicDataset(tokenizer, train,128)
valid_dataset = ToxicDataset(tokenizer,valid,128)

In [42]:
%%time
train_dataset[1]

CPU times: user 456 µs, sys: 0 ns, total: 456 µs
Wall time: 313 µs


{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([2625,   10,   25,  225,    5,  738,   10,   25,  225,    5,    1,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0

In [43]:
len(train_dataset), len(valid_dataset)

(21984, 5497)

In [44]:
# cach the dataset, so we can load it directly for training

torch.save(train_dataset, 'train_data.pt')
torch.save(valid_dataset, 'valid_data.pt')

For more details on how to use the nlp library check out this [notebook](https://colab.research.google.com/github/huggingface/nlp/blob/master/notebooks/Overview.ipynb).

## Write training script

In [45]:
import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, List, Optional

import numpy as np
import torch

from transformers import T5ForConditionalGeneration, T5Tokenizer, EvalPrediction
from transformers import (
    HfArgumentParser,
    DataCollator,
    Trainer,
    TrainingArguments,
    set_seed,
)


logger = logging.getLogger(__name__)

# prepares lm_labels from target_ids, returns examples with keys as expected by the forward method
# this is necessacry because the trainer directly passes this dict as arguments to the model
# so make sure the keys match the parameter names of the forward method
@dataclass
class T2TDataCollator():
    def collate_batch(self, batch: List) -> Dict[str, torch.Tensor]:
        """
        Take a list of samples from a Dataset and collate them into a batch.
        Returns:
            A dictionary of tensors
        """
        input_ids = torch.stack([example['input_ids'] for example in batch])
        lm_labels = torch.stack([example['target_ids'] for example in batch])
        lm_labels[lm_labels[:, :] == 0] = -100
        attention_mask = torch.stack([example['attention_mask'] for example in batch])
        decoder_attention_mask = torch.stack([example['target_attention_mask'] for example in batch])
        

        return {
            'input_ids': input_ids, 
            'attention_mask': attention_mask,
            'lm_labels': lm_labels, 
            'decoder_attention_mask': decoder_attention_mask
        }


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )

@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """
    train_file_path: Optional[str] = field(
        default='train_data.pt',
        metadata={"help": "Path for cached train dataset"},
    )
    valid_file_path: Optional[str] = field(
        default='valid_data.pt',
        metadata={"help": "Path for cached valid dataset"},
    )
    max_len: Optional[int] = field(
        default=512,
        metadata={"help": "Max input length for the source text"},
    )
    target_max_len: Optional[int] = field(
        default=32,
        metadata={"help": "Max input length for the target text"},
    )


def main():
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))

    # we will load the arguments from a json file, 
    #make sure you save the arguments in at ./args.json
    model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath('args.json'))

    if (
        os.path.exists(training_args.output_dir)
        and os.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
        )

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.local_rank,
        training_args.device,
        training_args.n_gpu,
        bool(training_args.local_rank != -1),
        training_args.fp16,
    )
    logger.info("Training/evaluation parameters %s", training_args)

    # Set seed
    set_seed(training_args.seed)

    # Load pretrained model and tokenizer
    #
    # Distributed training:
    # The .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.

    tokenizer = T5Tokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
    model = T5ForConditionalGeneration.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )

    # Get datasets
    print('loading data')
    train_dataset  = torch.load(data_args.train_file_path)
    valid_dataset = torch.load(data_args.valid_file_path)
    print('loading done')

    # Initialize our Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=T2TDataCollator(),
        prediction_loss_only=True
    )

    # Training
    if training_args.do_train:
        trainer.train(
            model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
        )
        trainer.save_model()
        # For convenience, we also re-save the tokenizer to the same directory,
        # so that you can share your model easily on huggingface.co/models =)
        if trainer.is_world_master():
            tokenizer.save_pretrained(training_args.output_dir)

    # Evaluation
    results = {}
    if training_args.do_eval and training_args.local_rank in [-1, 0]:
        logger.info("*** Evaluate ***")

        eval_output = trainer.evaluate()

        output_eval_file = os.path.join(training_args.output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results *****")
            for key in sorted(eval_output.keys()):
                logger.info("  %s = %s", key, str(eval_output[key]))
                writer.write("%s = %s\n" % (key, str(eval_output[key])))
    
        results.update(eval_output)
    
    return results


def _mp_fn(index):
    # For xla_spawn (TPUs)
    main()

In [46]:
import json
args_dict = {
  "num_cores": 8,
  "model_name_or_path": 't5-base',
  "max_len": 128 ,
  "target_max_len": 2,
  "output_dir": './models/tpu',
  "overwrite_output_dir": True,
  "per_device_train_batch_size": 4,
  "per_gpu_eval_batch_size": 4,
  "gradient_accumulation_steps": 4,
  "learning_rate": 3e-5,
  "tpu_num_cores": 8,
  "num_train_epochs": 16,
  "do_train": True
}
with open('args.json', 'w') as f:
  json.dump(args_dict, f)

In [47]:
import torch_xla.distributed.xla_multiprocessing as xmp


In [48]:
xmp.spawn(_mp_fn, args=(), nprocs=8, start_method='fork')

06/21/2020 15:31:56 - INFO - transformers.training_args -   PyTorch: setting up devices
06/21/2020 15:31:56 - WARNING - __main__ -   Process rank: -1, device: xla:1, n_gpu: 0, distributed training: False, 16-bits training: False
06/21/2020 15:31:56 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='./models/tpu', overwrite_output_dir=True, do_train=True, do_eval=False, do_predict=False, evaluate_during_training=False, per_device_train_batch_size=4, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=4, gradient_accumulation_steps=4, learning_rate=3e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=16, max_steps=-1, warmup_steps=0, logging_dir='runs/Jun21_15-31-56_e78b803b46ca', logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', local_rank=-1, tpu_num_cores=8, tpu_metrics_debug=False, dataloader_drop_last=

loading data


06/21/2020 15:32:04 - INFO - transformers.modeling_utils -   Weights of T5ForConditionalGeneration not initialized from pretrained model: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']


loading data


06/21/2020 15:32:04 - INFO - transformers.modeling_utils -   Weights of T5ForConditionalGeneration not initialized from pretrained model: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']


...

06/21/2020 16:17:55 - INFO - transformers.trainer -   {'loss': 0.07275704814454002, 'learning_rate': 8.07017543859649e-06, 'epoch': 11.692867540029113, 'step': 2000}
06/21/2020 16:17:55 - INFO - transformers.trainer -   Saving model checkpoint to ./models/tpu/checkpoint-2000
06/21/2020 16:17:55 - INFO - transformers.trainer -   {'loss': 0.06910901108443614, 'learning_rate': 8.07017543859649e-06, 'epoch': 11.692867540029113, 'step': 2000}
06/21/2020 16:17:55 - INFO - transformers.trainer -   Saving model checkpoint to ./models/tpu/checkpoint-2000
06/21/2020 16:17:55 - INFO - transformers.trainer -   {'loss': 0.07371945340337516, 'learning_rate': 8.07017543859649e-06, 'epoch': 11.692867540029113, 'step': 2000}
06/21/2020 16:17:55 - INFO - transformers.trainer -   {'loss': 0.06500033761809937, 'learning_rate': 8.07017543859649e-06, 'epoch': 11.692867540029113, 'step': 2000}
06/21/2020 16:17:55 - INFO - transformers.trainer -   Saving model checkpoint to ./models/tpu/checkpoint-2000
06/21/

06/21/2020 16:26:39 - INFO - transformers.trainer -   {'loss': 0.05868402540703937, 'learning_rate': 2.587719298245614e-06, 'epoch': 14.617176128093158, 'step': 2500}
06/21/2020 16:26:39 - INFO - transformers.trainer -   {'loss': 0.05936922314996264, 'learning_rate': 2.587719298245614e-06, 'epoch': 14.617176128093158, 'step': 2500}
06/21/2020 16:26:39 - INFO - transformers.trainer -   Saving model checkpoint to ./models/tpu/checkpoint-2500
06/21/2020 16:26:39 - INFO - transformers.trainer -   Saving model checkpoint to ./models/tpu/checkpoint-2500
06/21/2020 16:26:39 - INFO - transformers.trainer -   {'loss': 0.060925114739062335, 'learning_rate': 2.587719298245614e-06, 'epoch': 14.617176128093158, 'step': 2500}
06/21/2020 16:26:39 - INFO - transformers.trainer -   Saving model checkpoint to ./models/tpu/checkpoint-2500
06/21/2020 16:26:39 - INFO - transformers.trainer -   {'loss': 0.0621271152665322, 'learning_rate': 2.587719298245614e-06, 'epoch': 14.617176128093158, 'step': 2500}
06

06/21/2020 16:31:07 - INFO - transformers.trainer -   

Training completed. Do not forget to share your model on huggingface.co/models =)


06/21/2020 16:31:07 - INFO - transformers.trainer -   

Training completed. Do not forget to share your model on huggingface.co/models =)

06/21/2020 16:31:07 - INFO - transformers.trainer -   

Training completed. Do not forget to share your model on huggingface.co/models =)

06/21/2020 16:31:07 - INFO - transformers.trainer -   

Training completed. Do not forget to share your model on huggingface.co/models =)

06/21/2020 16:31:07 - INFO - transformers.trainer -   Saving model checkpoint to ./models/tpu


06/21/2020 16:31:07 - INFO - transformers.trainer -   

Training completed. Do not forget to share your model on huggingface.co/models =)



06/21/2020 16:31:07 - INFO - transformers.trainer -   Saving model checkpoint to ./models/tpu
06/21/2020 16:31:07 - INFO - transformers.trainer -   Saving model checkpoint to ./models/tpu
06/21/2020 16:31:0

06/21/2020 16:31:07 - INFO - transformers.trainer -   

Training completed. Do not forget to share your model on huggingface.co/models =)


06/21/2020 16:31:07 - INFO - transformers.trainer -   Saving model checkpoint to ./models/tpu


06/21/2020 16:31:07 - INFO - transformers.trainer -   

Training completed. Do not forget to share your model on huggingface.co/models =)


06/21/2020 16:31:07 - INFO - transformers.trainer -   Saving model checkpoint to ./models/tpu
06/21/2020 16:31:07 - INFO - transformers.trainer -   

Training completed. Do not forget to share your model on huggingface.co/models =)


06/21/2020 16:31:07 - INFO - transformers.trainer -   Saving model checkpoint to ./models/tpu
06/21/2020 16:31:07 - INFO - transformers.configuration_utils -   Configuration saved in ./models/tpu/config.json
06/21/2020 16:31:14 - INFO - transformers.modeling_utils -   Model weights saved in ./models/tpu/pytorch_model.bin
06/21/2020 16:31:14 - INFO - transformers.modeling_utils -   Model weights saved in ./models/tpu/pytorch_model.bin
06/21/2020 16:31:14 - INFO - transformers.modeling_utils -   Model weights saved in ./models/tpu/pytorch_model.bin
06/21/2020 16:31:14 - INFO - transformers.modeling_utils -   Model weight

Let's write the arguments in a dict and store in a json file. The above code will load this file and parse the arguments.

In [50]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm

from transformers import T5ForConditionalGeneration, T5Tokenizer

from tqdm.auto import tqdm

model = T5ForConditionalGeneration.from_pretrained('models/tpu').to('cpu')

In [51]:
valid_dataset = torch.load('valid_data.pt')
dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=32)
answers = []
for batch in tqdm(dataloader):
  outs = model.generate(input_ids=batch['input_ids'], 
                        attention_mask=batch['attention_mask'],
                        max_length=2,
                        early_stopping=True)
  outs = [tokenizer.decode(ids) for ids in outs]
  answers.extend(outs)
predictions = []
references = []
for ref, pred in zip(valid_dataset, answers):
  predictions.append(pred)
  references.append(tokenizer.decode(ref['target_ids']))

In [54]:
predictions[0], references[0]

('positive', 'positive')

In [53]:
from sklearn.metrics import classification_report
print(classification_report(references, predictions))

              precision    recall  f1-score   support

    negative       0.95      0.89      0.92      1562
     neutral       0.90      0.97      0.93      2230
    positive       0.96      0.92      0.94      1705

    accuracy                           0.93      5497
   macro avg       0.94      0.92      0.93      5497
weighted avg       0.93      0.93      0.93      5497

